<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [53]:
#indicator = pd.DataFrame(df, index=df.index)

In [54]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [55]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [56]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [57]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [58]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [59]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [60]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [61]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [62]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [63]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2017,1,1)
  end = dt.datetime.now()
  #end = dt.datetime(2018,1,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [64]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'YNDX'])

In [65]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [66]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [67]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
# to be discussed
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>middle_band):
        if adj_close>upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = 50
    middle_band1 = 55
    rsi = row['RSI']

    if (rsi>middle_band):
        if rsi>upper_band:
            score = ((rsi/upper_band)-1)*(-1)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                score = ((rsi/((upper_band-middle_band1)*0.5+middle_band1))-1)*(-1)
    else:
        if (rsi<lower_band):
            score = ((rsi/lower_band)-1)*(-1)
        else:
            score = (rsi/middle_band)-1

    return score
# To Add
# RSI & CCI

In [68]:
# to be discussed with Dustin 

lower_band = 30
upper_band = 70
middle_band = 50
middle_band1 = 55
rsi = 45

if (rsi>middle_band):
    if rsi>upper_band:
        score = ((rsi/upper_band)-1)*(-1)
    else:
        if rsi<middle_band1:
            score = 0
        else:
            score = ((rsi/((upper_band-middle_band1)*0.5+middle_band1))-1)*(-1)
else:
    if (rsi<lower_band):
        score = ((rsi/lower_band)-1)*(-1)
    else:
        score = (rsi/middle_band)-1

print(score)

-0.09999999999999998


In [69]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)

    score = ema_scr+(macd_signalline_scr*10)+bollinger_scr+sma_scr+rsi_scr
    print(stock, ema_scr, (macd_signalline_scr*10), bollinger_scr, sma_scr, score, '\n')
    return score

In [70]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.014511468767567592 -0.03317639378506487 -0.005903675415321552 -0.019911298670213795 -0.23917566240049792 

AAPL 0.011865350394713925 -0.0357382500689396 -0.003104156260030244 -0.019376611947409472 -0.115295234417775 

SAP 0.03959605186429127 -0.022966910056967382 -0.0006134783006990219 0.013714226061101398 0.1075008867014553 

ACN 0.05835667003246847 -0.05180076428277669 -0.007359539021774886 -0.015382126581858113 -0.15175734260237217 

VOW.DE 0.12326263053942088 0.03974649774189931 0.004954137691183358 0.009337460710257872 0.2362284359618051 

PLUG -0.20409855873393834 0.32588652767375437 0.037827607778149086 0.0675257425504443 0.34595044890908366 

MDO.DE 0.03305894645289564 -0.03357071453100824 -0.00856542901554469 -0.014361970731055385 -0.3133141233576273 

PYPL 0.020233863738835067 -0.022477638862732352 0.043553986213991536 -0.01775494809164324 -0.015608772866104181 

SQ -0.010241214800826293 -0.13688759487879398 -0.06934936748509213 -0.11124160047069998 -0.6526398981272106

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-05-20,28.629999,26.590000,28.200001,27.469999,39058400.0,27.469999,PLUG,26.685263,33.528351,25.732400,42.357500,26.009707,20.103754,31.090246,-1.371486,-2.266696,55.074429,100.684503,False,0.345950
2021-05-21,274.399994,271.399994,272.000000,272.399994,10975.0,272.399994,VOW.DE,269.166791,239.629436,269.880000,231.566001,268.649042,253.216601,278.863398,-1.901175,-2.983870,58.817018,119.823629,False,0.236228
2021-05-20,140.809998,137.809998,137.940002,140.169998,473800.0,140.169998,SAP,136.406157,131.210730,138.273682,128.799716,137.001983,133.295718,143.166137,1.363036,1.684963,57.639313,55.481426,False,0.107501
2021-05-20,64.809998,63.820000,63.820000,64.790001,1148300.0,64.790001,YNDX,63.785245,64.293842,63.464000,65.709500,63.807370,61.026981,67.093020,0.154198,0.056003,52.464287,36.230755,False,0.038226
2021-05-20,252.929993,245.350006,246.990005,251.539993,8102100.0,251.539993,PYPL,251.918803,246.922604,256.086801,253.623150,250.881521,232.172624,275.107379,-3.792939,-3.227537,48.041798,28.848995,False,-0.015609
2021-05-20,227.600006,225.240005,225.440002,226.440002,7474500.0,226.440002,V,225.989439,217.617350,228.055795,215.804129,226.374232,220.709957,236.543501,0.478367,1.312403,44.206414,-19.223596,False,-0.099634
2021-05-20,127.720001,125.099998,125.230003,127.309998,76756000.0,127.309998,AAPL,127.931496,126.431344,129.825577,128.902567,127.706418,121.586398,136.335741,-1.070136,-0.615153,46.552922,-11.004119,False,-0.115295
2021-05-20,285.130005,281.579987,281.700012,283.950012,2094700.0,283.950012,ACN,285.546855,269.802103,288.386002,267.826659,286.055247,280.604142,296.062863,0.351777,1.822660,43.221421,-62.141929,False,-0.151757
2021-05-20,3259.679932,3236.179932,3244.399902,3247.679932,2626900.0,3247.679932,AMZN,3269.571756,3222.804134,3313.659189,3224.288286,3266.967045,3100.119639,3506.308339,-18.360121,-7.585490,40.265212,-7.126800,False,-0.239176


In [71]:
df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
df_yd

<ipython-input-71-a76c1241de50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)


,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-05-20,3259.679932,3236.179932,3244.399902,3247.679932,2626900.0,3247.679932,AMZN,3269.571756,3222.804134,3313.659189,3224.288286,3266.967045,3100.119639,3506.308339,-18.360121,-7.585490,40.265212,-7.126800,False,1
2021-05-20,127.720001,125.099998,125.230003,127.309998,76756000.0,127.309998,AAPL,127.931496,126.431344,129.825577,128.902567,127.706418,121.586398,136.335741,-1.070136,-0.615153,46.552922,-11.004119,False,1
2021-05-20,140.809998,137.809998,137.940002,140.169998,473800.0,140.169998,SAP,136.406157,131.210730,138.273682,128.799716,137.001983,133.295718,143.166137,1.363036,1.684963,57.639313,55.481426,False,3
2021-05-20,285.130005,281.579987,281.700012,283.950012,2094700.0,283.950012,ACN,285.546855,269.802103,288.386002,267.826659,286.055247,280.604142,296.062863,0.351777,1.822660,43.221421,-62.141929,False,1
2021-05-21,274.399994,271.399994,272.000000,272.399994,10975.0,272.399994,VOW.DE,269.166791,239.629436,269.880000,231.566001,268.649042,253.216601,278.863398,-1.901175,-2.983870,58.817018,119.823629,False,4
2021-05-20,28.629999,26.590000,28.200001,27.469999,39058400.0,27.469999,PLUG,26.685263,33.528351,25.732400,42.357500,26.009707,20.103754,31.090246,-1.371486,-2.266696,55.074429,100.684503,False,3
2021-05-21,191.000000,190.100006,191.000000,190.100006,46.0,190.100006,MDO.DE,191.684774,185.550665,192.869999,182.348548,191.742362,188.335710,196.919289,-0.378944,0.259235,35.506252,-44.933578,False,1
2021-05-20,252.929993,245.350006,246.990005,251.539993,8102100.0,251.539993,PYPL,251.918803,246.922604,256.086801,253.623150,250.881521,232.172624,275.107379,-3.792939,-3.227537,48.041798,28.848995,False,1
2021-05-20,206.690002,202.199997,202.589996,204.910004,8316300.0,204.910004,SQ,223.268799,225.579002,230.557600,234.300499,220.179299,185.408985,267.256015,-11.138743,-8.333779,33.753994,-51.386380,False,0


In [72]:
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,
2018-03-28,1455.900024,1386.170044,1447.000000,1431.420044,13705300.0,1431.420044,AMZN,1516.794251,1347.842683,1537.284004,1320.036298,1524.466760,1457.784937,1632.253088,10.869706,30.461695,32.818825,-186.543374,True
2018-03-29,1455.469971,1365.199951,1406.000000,1447.339966,12581100.0,1447.339966,AMZN,1511.451613,1349.812927,1535.764004,1323.567498,1517.121351,1447.654534,1637.772492,3.806527,25.130661,34.145724,-139.573384,True
2018-10-11,1755.400024,1685.099976,1724.000000,1719.359985,13935900.0,1719.359985,AMZN,1906.378387,1816.663306,1931.987607,1823.597002,1899.703647,1769.755706,2074.403315,-34.746162,-6.476098,19.601718,-154.132505,True
2018-12-21,1480.000000,1363.959961,1464.989990,1377.449951,13640300.0,1377.449951,AMZN,1592.245094,1698.288567,1592.662397,1785.014905,1577.330174,1412.450752,1795.472246,-51.152443,-27.780822,23.797412,-135.643586,True
2020-02-28,1889.760010,1811.130005,1814.630005,1883.750000,9481000.0,1883.750000,AMZN,2006.136906,1898.192754,2021.518403,1852.041097,2013.641686,1883.394623,2243.210382,4.545262,40.575703,24.214313,-138.315342,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-24,27.650000,25.799999,27.480000,25.809999,5326300.0,25.809999,YNDX,31.142440,33.118246,32.024800,33.766000,30.839374,26.085076,37.365924,-1.261947,-0.418714,26.762105,-111.098247,True
2019-10-11,32.630001,28.910000,31.740000,29.990000,34452800.0,29.990000,YNDX,35.550212,36.594021,36.039200,37.406200,35.289838,32.246014,39.027985,-0.904930,-0.596082,17.201481,-211.522256,True
2020-03-12,34.000000,32.029999,32.330002,32.119999,4443900.0,32.119999,YNDX,40.769313,41.274374,42.606800,40.958400,40.045020,32.551380,49.809620,-2.679165,-1.776752,24.715394,-145.265994,True


In [73]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()